In [1]:
from ise.descriptors.detectanddescribe import DetectAndDescribe
from ise.indexer.featureindexer import FeatureIndexer
from ise.descriptors.rootsift import RootSIFT
from imutils import paths
import argparse, imutils, cv2, glob, os

In [2]:
detector = cv2.xfeatures2d.SIFT_create()
descriptor = RootSIFT()
dad = DetectAndDescribe(detector, descriptor)

In [3]:
fi = FeatureIndexer("features.hdf5",estNumImages=500, maxBufferSize = 5000)

In [4]:
filelist = glob.glob("/Users/Satish/Downloads/kaggle-fish/data/*")
filelist = [glob.glob(filelist[i]+"/*") for i in range(len(filelist))]

In [5]:
import itertools
filelist = list(itertools.chain(*filelist))
print (filelist[0])

/Users/Satish/Downloads/kaggle-fish/data/ALB/img_00003.jpg


In [6]:
for (i, imagePath) in enumerate(filelist):
    # check to see if progress should be displayed
    if i > 0 and i % 10 == 0:
        fi._debug("processed {} images".format(i), msgType="[PROGRESS]")

        #Extract the filename from the image path, then load the image itself
        filename = imagePath.rsplit("/")[-2]+"_"+imagePath.rsplit("/")[-1]
        image = cv2.imread(imagePath)
        image = imutils.resize(image, width = 320)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        #describe the image
        (kps, decs) = detector.detectAndCompute(image, None)
        kps = [kps[i].pt for i in range(len(kps))]

        # if either the keypoints or descriptors is None, then ignore the image
        if kps is None or decs is None:
            continue

        # index the features
        fi.add(filename, kps, decs)

#finish the indexing process
fi.finish()

[PROGRESS]processed 10 images - 2017-01-25 15:58:38.807021
[PROGRESS]processed 20 images - 2017-01-25 15:58:38.895795
[PROGRESS]processed 30 images - 2017-01-25 15:58:38.976898
[PROGRESS]processed 40 images - 2017-01-25 15:58:39.060279
[PROGRESS]processed 50 images - 2017-01-25 15:58:39.136095
[PROGRESS]processed 60 images - 2017-01-25 15:58:39.204600
[PROGRESS]processed 70 images - 2017-01-25 15:58:39.268006
[PROGRESS]processed 80 images - 2017-01-25 15:58:39.359596
[PROGRESS]processed 90 images - 2017-01-25 15:58:39.448260
[PROGRESS]processed 100 images - 2017-01-25 15:58:39.524855
[PROGRESS]processed 110 images - 2017-01-25 15:58:39.594722
[PROGRESS]processed 120 images - 2017-01-25 15:58:39.650442
[PROGRESS]processed 130 images - 2017-01-25 15:58:39.729962
[PROGRESS]processed 140 images - 2017-01-25 15:58:39.829004
[INFO]initial buffer full - 2017-01-25 15:58:39.894983
[INFO]create datasets... - 2017-01-25 15:58:39.895164
[INFO]writing `image_ids` buffer - 2017-01-25 15:58:39.89826

In [10]:
import h5py
import numpy as np

In [11]:
db = h5py.File("features.hdf5",mode="r")

In [12]:
for key in db.keys():
    print (key)

features
image_ids
index


In [13]:
db["image_ids"].shape

(377,)

In [14]:
db["index"].shape

(377, 2)

In [15]:
db["image_ids"][376]

'YFT_img_07849.jpg'

In [16]:
db["features"].shape[0]

141844

In [17]:
from ise.ir.vocabulary import Vocabulary
import argparse 

In [18]:
voc = Vocabulary("features.hdf5")
vocab = voc.fit(64, 0.25)

[INFO] start sampling .. - 2017-01-25 17:06:25.044720
[INFO] sampled35,461 features from a population of 141,844 - 2017-01-25 17:07:05.747796
[INFO] clustering with k-64 - 2017-01-25 17:07:05.747934
[INFO] cluster shape: (64, 128) - 2017-01-25 17:07:07.733402


In [19]:
vocab[0]

array([  14.25688073,   19.09174312,   22.00611621,   18.96636086,
         19.55198777,   17.4266055 ,   16.33486239,   12.83486239,
         32.47400612,   30.73853211,   23.35015291,   18.4969419 ,
         17.7706422 ,   14.48929664,   17.59480122,   26.79816514,
         42.82415902,   23.10091743,   14.38379205,   11.04587156,
         13.6941896 ,   16.70642202,   31.79357798,   60.02752294,
         25.09785933,   13.89908257,    8.35626911,    9.25076453,
         11.39602446,   25.12844037,   39.50764526,   47.61926606,
         13.74006116,   16.44801223,   23.38685015,   22.13608563,
         29.01223242,   36.24464832,   25.7706422 ,   16.1146789 ,
         55.90061162,   40.36850153,   34.34556575,   23.64220183,
         20.28287462,   23.01987768,   19.5412844 ,   25.75535168,
        112.85321101,   48.85626911,   18.22018349,    7.56880734,
          5.86697248,   11.38685015,   27.21406728,   72.50917431,
         34.61620795,   19.36238532,   16.68960245,   17.66513

In [20]:
vocab.shape

(64, 128)

In [21]:
len(vocab)

64

In [22]:
import numpy as np
vis = {i:[] for i in np.arange(0, len(vocab))}

In [37]:
vis

{0: [(255.91638358389332,
   array([ 190.68371582,   54.47125244]),
   'ALB_img_06850.jpg'),
  (256.5929432114695,
   array([ 152.17181396,   24.26014328]),
   'ALB_img_06058.jpg'),
  (261.90196996173756,
   array([ 191.34091187,   50.10915756]),
   'ALB_img_02025.jpg'),
  (263.57272375524627,
   array([ 80.98532867,   9.54574203]),
   'ALB_img_04979.jpg'),
  (263.59091585604125,
   array([ 235.2525177 ,   54.40200806]),
   'NoF_img_04509.jpg'),
  (264.60490939850285,
   array([ 299.25558472,  115.45253754]),
   'YFT_img_02897.jpg'),
  (266.36322887850906,
   array([ 31.38042068,  99.29618835]),
   'NoF_img_07641.jpg'),
  (269.56828035208349,
   array([ 256.66912842,   69.32185364]),
   'NoF_img_06451.jpg'),
  (271.17859136403331,
   array([ 175.40797424,  112.21047211]),
   'ALB_img_06247.jpg'),
  (272.10883262655443,
   array([ 281.88870239,   49.44208145]),
   'ALB_img_06247.jpg'),
  (272.11153548021173,
   array([ 124.66247559,   31.50930023]),
   'YFT_img_06980.jpg'),
  (273.28496

In [30]:
from sklearn.metrics import pairwise
for (i, imageID) in enumerate(db["image_ids"]):
    (start, end) = db["index"][i]
    rows = db["features"][start:end]
    (kps,descs)=(rows[:,:2],rows[:,2:])

    # loop over each of the individual keypoints and feature vectors
    for (kp, features) in zip(kps, descs):
        # compute the distance between the feature vector and all clusters, meaning that we'll have one distance value for each cluster
        D = pairwise.euclidean_distances(features.reshape(1,-1), Y=vocab)[0]

        #loop over the distances dictionary
        for j in np.arange(0, len(vocab)):
            #grab the set of top visualization results for the current visual word and update the top results with the tuple of the distance, keypoint and imageID
            topResults = vis.get(j)
            topResults.append((D[j], kp, imageID))

            # sort the top results list by their distance, keeping only the best 16, then update the visualization dictionary
            topResults = sorted(topResults, key=lambda r:r[0])[:16]
            vis[j] = topResults


In [27]:
db["features"][1].shape

(130,)

In [32]:
topResults[0]

(211.4145101058607, array([  98.35342407,  193.77241516]), 'YFT_img_06434.jpg')

In [36]:
vis.get(0)

[(255.91638358389332,
  array([ 190.68371582,   54.47125244]),
  'ALB_img_06850.jpg'),
 (256.5929432114695,
  array([ 152.17181396,   24.26014328]),
  'ALB_img_06058.jpg'),
 (261.90196996173756,
  array([ 191.34091187,   50.10915756]),
  'ALB_img_02025.jpg'),
 (263.57272375524627,
  array([ 80.98532867,   9.54574203]),
  'ALB_img_04979.jpg'),
 (263.59091585604125,
  array([ 235.2525177 ,   54.40200806]),
  'NoF_img_04509.jpg'),
 (264.60490939850285,
  array([ 299.25558472,  115.45253754]),
  'YFT_img_02897.jpg'),
 (266.36322887850906,
  array([ 31.38042068,  99.29618835]),
  'NoF_img_07641.jpg'),
 (269.56828035208349,
  array([ 256.66912842,   69.32185364]),
  'NoF_img_06451.jpg'),
 (271.17859136403331,
  array([ 175.40797424,  112.21047211]),
  'ALB_img_06247.jpg'),
 (272.10883262655443,
  array([ 281.88870239,   49.44208145]),
  'ALB_img_06247.jpg'),
 (272.11153548021173,
  array([ 124.66247559,   31.50930023]),
  'YFT_img_06980.jpg'),
 (273.28496565101904,
  array([ 104.64009857,   

In [ ]:
from ise.resultsmontage import ResultsMontage